In [1]:
import time
import re
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [5]:
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
# driver.implicitly_wait(10)

In [6]:
driver.get('https://hsreplay.net/meta/#tab=archetypes')
time.sleep(10)

# try:
#     element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "myDynamicElement"))
#     )
# finally:
#     driver.quit()

In [7]:
driver.execute_script('return document.readyState;')

'complete'

In [8]:
innerHTML = driver.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerHTML, 'html.parser')

In [ ]:
print(soup.prettify())

In [13]:
all_decks = soup.find_all("a",class_="table-cell", attrs={"aria-describedby": re.compile("column0")})

In [18]:
all_decks[0]

<a aria-describedby="table1-row0 table1-column0" class="table-cell" href="/archetypes/267/gonk-druid" role="gridcell" style="height: 36px; left: 0px; position: absolute; top: 0px; width: 172.667px; color: rgb(255, 80, 80); line-height: 36px; background: white;">37.5%</a>

In [17]:
all_decks[0].get('href')

'/archetypes/267/gonk-druid'

In [23]:
archetypes = list(map(lambda tag: (tag.get('href'), tag.contents[0].rstrip('%')), all_decks))

In [24]:
archetypes_f = list(filter(lambda archetype: archetype[0] is not None, archetypes))

In [27]:
links = [x[0] for x in archetypes_f]

In [28]:
links

['/archetypes/267/gonk-druid',
 '/archetypes/6/malygos-druid',
 '/archetypes/247/mechathun-druid',
 '/archetypes/226/mill-druid',
 '/archetypes/150/quest-druid',
 '/archetypes/205/spiteful-druid',
 '/archetypes/149/taunt-druid',
 '/archetypes/7/token-druid',
 '/archetypes/270/undatakah-druid',
 '/archetypes/272/big-secret-hunter',
 '/archetypes/139/deathrattle-hunter',
 '/archetypes/137/midrange-hunter',
 '/archetypes/212/odd-hunter',
 '/archetypes/180/recruit-hunter',
 '/archetypes/138/secret-hunter',
 '/archetypes/179/spell-hunter',
 '/archetypes/182/big-spell-mage',
 '/archetypes/145/elemental-mage',
 '/archetypes/1/freeze-mage',
 '/archetypes/235/murloc-mage',
 '/archetypes/214/odd-mage',
 '/archetypes/142/quest-mage',
 '/archetypes/273/secret-odd-mage',
 '/archetypes/11/tempo-mage',
 '/archetypes/261/deathrattle-paladin',
 '/archetypes/215/even-paladin',
 '/archetypes/268/holy-wrath-paladin',
 '/archetypes/249/mech-paladin',
 '/archetypes/168/otk-dk-paladin',
 '/archetypes/216/odd

In [19]:
all_decks[0].contents[0].rstrip('%')

'37.5%'

In [29]:
deck_name_extract = re.compile('[^\/]+$')

In [ ]:
for link in links:
    

In [8]:
all_decks = soup.find_all("li", class_="archetype-list-item")
links = list(map(lambda tag: tag.find("a").get('href'), all_decks))

deck_name_extract = re.compile('[^\/]+$')
base_url = 'https://hsreplay.net'
winrates = {}
df_wr = pd.DataFrame()

for link in links[:3]:
    deck = deck_name_extract.search(link).group()
    url = base_url + link + '#tab=matchups'
    print(f'{deck}:{url}')
    driver.get(url)
    time.sleep(5)
    innerHTML = driver.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML, 'html.parser')
    matchups = soup.find_all("a",class_="table-cell", attrs={"aria-describedby": re.compile("column0")})
    winrates[deck] = list(map(lambda matchup: (float(matchup.contents[0].rstrip('%')), deck_name_extract.search(matchup.get('href')).group()), matchups))
    
    zlst = list(zip(*winrates[deck]))
    s_wr = pd.Series(zlst[0], index=zlst[1])
    
    df_wr[deck] = s_wr

In [16]:
driver.quit()

df_wr = df_wr.T
df_wr = df_wr[list(df_wr.index.values)]
df_wr.to_csv(f"hsreplay_winrates_{datetime.today().strftime('%Y%m%d')}.csv")

In [ ]:
# File approach
# for i, line in enumerate(lines):
#     if line.startswith('Versus'):
#         matchups.append((' '.join(lines[i-1].split()),' '.join(lines[i].split()[1:]), float(' '.join(lines[i+1].split()[1:]).rstrip('%'))))